In [1]:
%pip install sktime==0.11.3
#%pip install numpy=1.20

Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np
import pandas as pd
import timeit
from sklearn.model_selection import RepeatedKFold
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import ShuffleSplit
from sklearn.metrics import fbeta_score, make_scorer
import tensorflow as tf
from pickle import dump

from sktime.classification.hybrid import HIVECOTEV1
from sktime._contrib.vector_classifiers._rotation_forest import RotationForest
from sktime.datasets import load_from_arff_to_dataframe as load_arff

In [3]:
#%tensorflow_version 2.x
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [4]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print(
      '\n\nThis error most likely means that this notebook is not '
      'configured to use a GPU.  Change this in Notebook Settings via the '
      'command palette (cmd/ctrl-shift-P) or the Edit menu.\n\n')
  raise SystemError('GPU device not found')

In [5]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=10)
X, y = load_arff("AllCandidas_TRAIN_V2.arff")
print(np.shape(X))
print(type(X))
precision = []
accuracy = []
f1 = []
recall = []
def gpu(X, y, kf):
  with tf.device('/device:GPU:0'):
    i= 0
    for train_index, test_index in kf.split(X):
        #print("TRAIN:", train_index, "TEST:", test_index)    
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y[train_index], y[test_index]
        #print(X_train.head())
        #print(type(X_train))
        HC1 =  HIVECOTEV1(
                stc_params={
                    "estimator": RotationForest(n_estimators=3),
                    "n_shapelet_samples": 100,
                    "max_shapelets": 10,
                    "batch_size": 20,
                },
                tsf_params={"n_estimators": 3},
                rise_params={"n_estimators": 3},
                cboss_params={"n_parameter_samples":1, "max_ensemble_size": 1, "min_window":1, "max_win_len_prop":1},
                verbose=10
            )
        print(f"Interaction {i}")
        print("In trainning...")
        HC1.fit(X_train, y_train)
        print("In prediction...")
        i=i+1
        
        # Predict and print accuracy
        predictions = HC1.predict(X_test)
        precision.append(precision_score(y_test, predictions, average='macro',zero_division=1))
        accuracy.append(accuracy_score(y_test, predictions))
        recall.append(recall_score(y_test, predictions, average='macro',zero_division=1))
        f1.append(f1_score(y_test, predictions, average='macro'))
        #print("In prediction...")
        # Predict and print accuracy
       # predictions = HC2.predict(X_test)
    print(type(X_train))
    print(np.shape(X_train))
    print(f"Precision: {np.mean(precision)}, Accuracy:{np.mean(accuracy)}, Recall: {np.mean(recall)}, F1: {np.mean(f1)}")
    return np.mean(precision), np.mean(accuracy), np.mean(recall), np.mean(f1), np.std(accuracy), HC1

(12256, 1)
<class 'pandas.core.frame.DataFrame'>


In [6]:
if tf.config.list_physical_devices('GPU'):
  print("Using MirroredStrategy")
  strategy = tf.distribute.MirroredStrategy()
else:  # Use the Default Strategy
  print("Using Default Strategy")
  strategy = tf.distribute.get_strategy()

with strategy.scope():
  # Do something interesting
  print(tf.Variable(1.))
  precision, accuracy, recall, f1, std, HC1 = gpu(X,y,kf)

Using MirroredStrategy
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
MirroredVariable:{
  0: <tf.Variable 'Variable:0' shape=() dtype=float32, numpy=1.0>
}
Interaction 0
In trainning...
STC  18:47:24 14/05/2022
STC train estimate  18:47:25 14/05/2022
STC weight = 0.2520077400223754
TSF  18:47:26 14/05/2022
TSF train estimate  18:47:31 14/05/2022
TSF weight = 0.5807947701765465
RISE  18:47:35 14/05/2022
RISE train estimate  18:48:31 14/05/2022
RISE weight = 0.41858540622621815
cBOSS (estimate included) 18:56:12 14/05/2022
cBOSS weight = 0.07635030816859241
In prediction...
Interaction 1
In trainning...
STC  20:17:13 14/05/2022
STC train estimate  20:17:15 14/05/2022
STC weight = 0.23553061983745235
TSF  20:17:16 14/05/2022
TSF train estimate  20:17:21 14/05/2022
TSF weight = 0.6122984643166873
RISE  20:17:26 14/05/2022
RISE train estimate  20:18:27 14/05/2022
RISE weight = 0.5093177064321204
cBOSS (estimate included) 20:27:44 14/05

In [9]:
df_metrics = pd.DataFrame(data=np.zeros((1, 5), dtype=np.float), index=[0],
                       columns=['precision', 'accuracy', 'recall','f1_score','std'])
df_metrics['precision'] = precision
df_metrics['accuracy'] = accuracy
df_metrics['recall'] = recall
df_metrics['f1_score'] = f1
df_metrics['std'] = std
df_metrics

C:\Users\mlb\AppData\Local\Temp/ipykernel_33544/3617186161.py:1: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  df_metrics = pd.DataFrame(data=np.zeros((1, 5), dtype=np.float), index=[0],


,precision,accuracy,recall,f1_score,std
0,0.934222,0.934155,0.934524,0.933926,0.014876


In [10]:
#save metrics
print("Saving metrics...")
df_metrics.to_csv("results/" + 'df_metrics_HC1.csv', index=False)
#save the model
print("Saving model...")
dump(HC1, open('results/'+'hc1_model.pkl', 'wb'))
print("Metrics of model:")
df_metrics

Saving metrics...
Saving model...
Metrics of model:


,precision,accuracy,recall,f1_score,std
0,0.934222,0.934155,0.934524,0.933926,0.014876


In [ ]:
print('GPU (s):')
gpu_time = timeit.timeit('gpu()', number=10, setup="from __main__ import gpu")
print(gpu_time)
print('GPU speedup over CPU: {}x'.format(int(cpu_time/gpu_time)))